In [19]:
import  torch
from pathlib import Path
import requests
import pickle
import gzip
from matplotlib import pyplot as plt
from tqdm import tqdm

In [8]:
DATA_PATH  = Path("data")
PATH = DATA_PATH / "mnist"
PATH.mkdir(parents=True, exist_ok=True)

In [9]:
print("Downloading MNIST dataset...")
URL = "https://github.com/pytorch/tutorials/raw/main/_static/"
FILENAME = "mnist.pkl.gz"
if not (PATH/FILENAME).exists():
    content = requests.get(URL + FILENAME).content
    with (PATH/FILENAME).open("wb") as f:
        f.write(content)
print(f"{FILENAME} Download complete!!")

In [12]:
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [16]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

In [21]:
plt.imshow(x_train[49999].reshape(28,28),cmap='grey')

In [24]:
#convert data to torch tensor
x_train,y_train, x_valid,y_valid =  map(torch.tensor, (x_train,y_train,x_valid,y_valid))
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

In [27]:
#using nn.Functional
import torch.nn.functional as F 
loss_fn = F.cross_entropy

from torch import nn
class Mnist_LR(nn.Module):
    def __init__(self):
        super(Mnist_LR, self).__init__()
        self.linear = nn.Linear(in_features=784,out_features=10)
    def forward(self, x):
        return self.linear(x)


In [33]:
model = Mnist_LR()
list(model.parameters())
model.state_dict()
type(model)

In [46]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [47]:
def fit(model:Mnist_LR,epochs:int):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        y_preds = model(x_train)
        train_loss = loss_fn(y_preds,y_train)
        optimizer.zero_grad()
        train_loss.backward()
        
        optimizer.step()
        
        with torch.inference_mode():
            y_val_preds = model(x_valid)
            val_loss = loss_fn(y_val_preds, y_valid)
            val_losses.append(val_loss)
        if epoch % 10 == 0:
            print(f"Epoch:{epoch} | Train loss:{train_loss} | Test loss: {val_loss}")        

In [48]:
fit(model= model,epochs=10000)